# Домашняя работа 2

## Регрессия

С 1ого октября отменяется НДС на бриллианты, теперь они становятся новым инвестиционным инструментом. Давайте сделаем модель ценообразования для них.

Скачайте датасет diamonds.csv

В нем представлены характеристики бриллиантов и их цены с сайта jamesallen (B2C площадка) с 2022-07-01

**Описание полей**


* fluor - флуорисценуия (свойство камня светиться в лучах ультр)
* symmetry - показатель симметричности
* platform - название платформы, где был размещен камень
* quality_group - составной показатель из cut polish symmetry
* size_group - каратно весовая группа
* big_size_group - каратно-весовая группа
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только у круглых камней)
* polish - полировка
* id - номер камня
* date - дата
* price - цена
* carat - кол-во карат
* tablepercent - размер площадки по отношению ширине
* price_per_carat - цена за карат
* z - длина (diameter)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота



In [4]:
import pandas as pd
import numpy as np

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

# path = 'drive/MyDrive/'

In [6]:
df = pd.read_csv('/content/diamonds.csv', index_col=0)
df

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NONE,EX,jamesallen,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
48477,MED,EX,jamesallen,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
236786,NONE,EX,jamesallen,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
235781,NONE,EX,jamesallen,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
277744,NONE,VG,jamesallen,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20595,NONE,EX,jamesallen,RD,D,VS2,EX,EX,7535786,202207,8700.0,0.97,8969.07,6.36,6.32,62.2,3.93104
282754,NONE,EX,jamesallen,RD,D,VS2,EX,EX,14631732,202206,6650.0,0.90,7388.89,6.13,6.18,61.7,3.81306
141550,NONE,EX,jamesallen,RD,J,SI2,EX,EX,13932645,202205,11570.0,1.52,7611.84,7.36,7.39,61.0,4.50790
71237,STG,EX,jamesallen,RD,G,VS1,EX,EX,12699310,202207,8220.0,1.09,7541.28,6.55,6.60,62.0,4.09200


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222222 entries, 135269 to 310690
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fluor            222207 non-null  object 
 1   symmetry         222218 non-null  object 
 2   platform         222218 non-null  object 
 3   shape            222218 non-null  object 
 4   color            222218 non-null  object 
 5   clarity          222218 non-null  object 
 6   cut              148981 non-null  object 
 7   polish           222218 non-null  object 
 8   id               222222 non-null  int64  
 9   date             222222 non-null  int64  
 10  price            222222 non-null  float64
 11  carat            222222 non-null  float64
 12  price_per_carat  222222 non-null  float64
 13  z                222222 non-null  float64
 14  x                222222 non-null  float64
 15  depth_perc       222222 non-null  float64
 16  y                222222 non-null 

In [8]:
df.nunique()

fluor                   4
symmetry                4
platform                1
shape                  10
color                  10
clarity                 9
cut                     3
polish                  3
id                 162278
date                    4
price                8437
carat                 632
price_per_carat     52605
z                    1116
x                     904
depth_perc            318
y                   37638
dtype: int64

Как мы видим признак 'platform' состоит всего из одного значения и имеет 4-е NaN, что скорее всего тоже являются тем же значением - 'jamesallen'. Поэтому было принято решение удалить данный признак, он малоинформативен. Его можно будет без всякого труда вернуть обратно, если в том будет необходимость, а пока он только мешает.

In [9]:
df.drop(columns='platform', inplace=True)

Необходимо предсказать `price_per_carat`

### Задание 1: Очистка

Не все камни успевают продаться за один месяц, поэтому в таблице есть повторы. Объедините данные по одному камню: подумайте, как лучше это сделать, какую цену брать.

**Попробуйте найти аномалии:** вдруг цена на некоторые камни сильно меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что остальные параметры камня не меняются.

In [10]:
# Посчитаем статистику по кол-ву камней

id_stat = df.groupby(by='id')['carat'].count().to_frame() \
.rename(columns={'carat': 'amount_of_diamonds'}) 

id_stat

,amount_of_diamonds
id,
0,4
19423,3
19449,2
19476,1
19501,3
...,...
15418480,1
15418482,1
15418486,1


In [11]:
# разделим выборку из исходного датасета на две части
# для начала создадим маски

count_1 = id_stat.query('amount_of_diamonds == 1')
count_2and3 = id_stat.query('amount_of_diamonds == [2, 3]')

In [12]:
# выберем те камни, которые встречались один раз

df_one = df.query('id in @count_1.index')

# и те камни, которые встречались 2 или 3 раза

df_three = df.query('id in @count_2and3.index')

In [13]:
# Посчитаем стандартное отклонение по нескольким полям,
# подумаем где оно должно равняться 0, а где меняться в каких-то разумных пределах

list_digit_columns = df.select_dtypes(exclude='object').columns

stat = df_three.groupby('id')[list_digit_columns[1:]].agg(np.std) 
# list_digit_columns без колонки id

stat

,date,price,carat,price_per_carat,z,x,depth_perc,y
id,,,,,,,,
19423,1.000000,334.863156,0.0,318.916742,0.0,0.0,0.404145,0.026835
19449,1.414214,494.974747,0.0,423.054916,0.0,0.0,0.000000,0.000000
19501,1.000000,831.384388,0.0,415.692194,0.0,0.0,0.346410,0.027436
19504,0.707107,0.000000,0.0,0.000000,0.0,0.0,0.212132,0.014213
19505,1.414214,813.172798,0.0,404.564074,0.0,0.0,0.353553,0.028602
...,...,...,...,...,...,...,...,...
15409696,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
15410303,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
15410668,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000


Первое что бросается в глаза это то, что у некоторых камней меняются размеры. В столбце 'y' (высота камня) в первых 5 строчках мы видим что высота у одного и того же камня может меняться. Посмотрим на изменения на примере первой строчки (id = 19423)

In [14]:
df_three[list_digit_columns].loc[df_three['id'] == 19423]

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
7,19423,202207,14550.0,1.05,13857.14,6.67,6.64,59.7,3.96408
5,19423,202205,13970.0,1.05,13304.76,6.67,6.64,59.0,3.91760
6,19423,202206,13970.0,1.05,13304.76,6.67,6.64,59.7,3.96408


Один и тот же камень "подрос"? Видим что и соотношение высоты к ширине ('depth_perc') ± справедливо изменилось. Но всё же, можем заключить, что такого быть не должно. Камни возможно могут стать меньше путём дополнительных обработок, но увеличиться они никак не могут.

Мы можем предположить, что произошла какая-то ревизия, при которой заново произвели измерения камней, где подругому сыграла погрешность измерения, либо замерили более новым и точным оборудованием.

Будем считать аномалией любое сильное изменение размеров камня.

Теперь проверим изменения камня в каратах ('carat')

In [15]:
stat[stat['carat'] > 0]

,date,price,carat,price_per_carat,z,x,depth_perc,y
id,,,,,,,,
6696046,1.414214,5161.879503,0.014142,3490.519488,0.056569,0.042426,0.494975,0.062756
10337722,1.000000,178.978583,0.040415,61.510898,0.057735,0.092376,0.351188,0.038917
13631105,0.707107,113.137085,0.007071,155.478639,0.000000,0.000000,0.141421,0.009687
13636926,0.707107,0.000000,0.014142,120.851620,0.000000,0.000000,0.141421,0.009560
14020956,1.000000,236.713610,0.034641,525.141309,0.005774,0.005774,1.039230,0.069213
14040944,0.707107,0.000000,0.035355,343.505403,0.021213,0.007071,0.070711,0.000028
14104780,1.414214,777.817459,0.021213,487.599623,0.028284,0.028284,0.848528,0.057502
14366972,1.414214,169.705627,0.035355,54.539146,0.028284,0.063640,0.212132,0.052842
14408689,0.707107,106.066017,0.070711,582.182226,0.035355,0.056569,1.909188,0.154128


Такие изменения есть у некоторых объектов, наряду с изменениями размеров.

Рассмотрим один из них, возьмём 9-ый объект (id = 14408689), поскольку там наблюдаются самое большое отклонение по 'depth_perc' и 'carat'

In [16]:
df_three[list_digit_columns].loc[df_three['id'] == 14408689]

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
228731,14408689,202206,5910.0,1.0,5910.00,6.17,6.23,64.9,4.04327
228732,14408689,202207,6060.0,0.9,6733.33,6.12,6.15,62.2,3.82530


Если верить предположению, что это просто по-новому измерили, то всё выглядит реалистично, кроме высоты (y), там изменение на ~ -0.2. В ювелирном деле погрешность измерения, или улучшение точности, вряд-ли находится в десятой доле, и скорей всего, даже не в стотысячной. Потому такое изменение можно считать аномальным

Хочется также посмотреть на первый объект по изменениям в каратах, у него самое большое отклонение в цене и цене за карат (id = 6696046)

In [17]:
df_three[list_digit_columns].loc[df_three['id'] == 6696046]

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
17866,6696046,202205,15650.0,1.54,10162.34,7.47,7.39,61.0,4.50790
17868,6696046,202207,22950.0,1.52,15098.68,7.39,7.45,61.7,4.59665


Если принять во внимание, что цифра которая находится в поле 'date', это скорей всего такой перевод даты в число, где каждая единица - это новый месяц (в задании говорилось, что не все бриллианты продаются за один МЕСЯЦ), то мы видим, что дата выросла на две единицы (условные два месяца). А цена и цена за карат выросли почти на 50%. За два месяца это очень крутой скачок, но теоретически такое возможно.

Отметим, что отклонения по дате - нормальны, поскольку не всегда удаётся продать товар сразу

Подобным образом рассмотрим длину (диаметр) и ширину камня

In [18]:
stat.query('z > 0 | x > 0')

,date,price,carat,price_per_carat,z,x,depth_perc,y
id,,,,,,,,
5062756,0.707107,403.050865,0.000000,200.521341,0.042426,0.042426,0.000000,0.160654
5476310,1.414214,155.563492,0.000000,154.021999,0.028284,0.028284,0.282843,0.035780
6696046,1.414214,5161.879503,0.014142,3490.519488,0.056569,0.042426,0.494975,0.062756
9840762,1.414214,162.634560,0.000000,86.507444,0.021213,0.021213,0.424264,0.046669
10337722,1.000000,178.978583,0.040415,61.510898,0.057735,0.092376,0.351188,0.038917
...,...,...,...,...,...,...,...,...
14675206,0.707107,56.568542,0.000000,56.568542,0.070711,0.070711,0.070711,0.049696
14733664,0.707107,381.837662,0.000000,272.738157,0.035355,0.035355,0.000000,0.021920
14788031,0.707107,63.639610,0.000000,60.613193,0.056569,0.056569,0.000000,0.035525


In [19]:
# рассмотрим первый объект

df_three[list_digit_columns].loc[df_three['id'] == 5062756]

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
10883,5062756,202205,15640.0,2.01,7781.09,8.07,8.01,62.0,4.9662
10884,5062756,202206,15070.0,2.01,7497.51,8.01,8.07,62.0,5.1934


In [20]:
# теперь второй объект

df_three[list_digit_columns].loc[df_three['id'] == 5476310]

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
11535,5476310,202205,5360.0,1.01,5306.93,6.35,6.31,63.0,3.9753
11537,5476310,202207,5580.0,1.01,5524.75,6.31,6.35,63.4,4.0259


In [21]:
# третий

df_three[list_digit_columns].loc[df_three['id'] == 6696046]

,id,date,price,carat,price_per_carat,z,x,depth_perc,y
17866,6696046,202205,15650.0,1.54,10162.34,7.47,7.39,61.0,4.50790
17868,6696046,202207,22950.0,1.52,15098.68,7.39,7.45,61.7,4.59665


Интересная картина, размеры перепутаны диагонально. Когда объект встречается второй раз, то его значение z оказывается в x, и наоборот. В третьем случае мы видим, что значения z всё-таки слегка изменилось, но всё-же, похоже на то, что данные меняются местами при повторном возникновении объекта.

Теперь, давайте посмотрим сколько объектов имели изменения по разным полям, отвечающим за размер камня 

In [22]:
print(f"Количество камней, которые изменились по каратам: \
{ len(stat[stat['carat'] > 0]) }")
print(f"Количество камней, которые изменились по диаметру: \
{ len(stat[stat['z'] > 0]) }")
print(f"Количество камней, которые изменились по ширине: \
{ len(stat[stat['x'] > 0]) }")
print(f"Количество камней, которые изменились по отношению высоты к длине: \
{ len(stat[stat['depth_perc'] > 0]) }")
print(f"Количество камней, которые изменились по высоте: \
{ len(stat[stat['y'] > 0]) }")
print(f'Общее количество камней в выборке: {len(stat)}')

Количество камней, которые изменились по каратам: 11
Количество камней, которые изменились по диаметру: 84
Количество камней, которые изменились по ширине: 82
Количество камней, которые изменились по отношению высоты к длине: 30352
Количество камней, которые изменились по высоте: 30366
Общее количество камней в выборке: 50373


До этого было сложно определить, является ли аномальным то или иное отклонение. 

Колонку 'depth_perc' можно никак не оценивать, поскольку, как мы могли наблюдать её бесперебойную точность, она скорей всего содержала в себе строгую формулу подсчёта высоты к ширине, от чего она была не причиной аномалий, а их следствием. Например, мы видим что количество камней по 'depth_perc' почти совпадает с количеством по 'y'. Любое изменение по высоте ('y') или ширине ('x') будет менять 'depth_perc'.

Что касается непосредственно самой высоты, то мы видели как она сильно изменялась (относительно ювелирной индустрии). Такое изменение сложно оправдать даже ревизией. Тоже самое касается изменения по каратам.

Начнём с удаления тех объектов, которые повторялись, но с изменением по высоте

In [23]:
# создадим переменную со списком тех индексов, высота в которых менялась

list_y_index = stat[stat['y'] > 0].index

In [24]:
# сохраняем тот объект, что встретился раньше

cleared_y_changes = df_three[df_three['id'].isin(list_y_index)].sort_values('date') \
.drop_duplicates('id', keep='first')

cleared_y_changes

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
303114,FNT,VG,RD,E,SI1,EX,EX,14701141,202205,6010.0,1.00,6010.00,6.32,6.37,63.0,4.01310
310231,NONE,VG,PR,H,VS1,EX,EX,14733282,202205,12670.0,1.50,8446.67,6.54,6.43,70.0,4.50100
19296,NONE,VG,PS,I,I1,NaN,VG,7178756,202205,2540.0,1.02,2490.20,8.77,5.48,63.0,3.45240
71746,FNT,G,EM,H,VS2,NaN,G,12757333,202205,3470.0,0.90,3855.56,6.27,4.77,68.0,3.24360
22150,NONE,G,EM,G,I1,NaN,VG,7831032,202205,1830.0,1.02,1794.12,6.98,4.84,64.0,3.09760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310395,NONE,VG,RD,H,VVS2,EX,EX,14733664,202206,13150.0,1.40,9392.86,7.16,7.11,62.0,4.40820
234790,NONE,EX,RD,J,I1,EX,EX,14433593,202206,14850.0,2.01,7388.06,8.14,8.08,61.5,4.96920
327897,STG,VG,RD,G,I1,EX,VG,14788031,202206,3220.0,1.05,3066.67,6.52,6.44,62.8,4.04432
228731,MED,VG,RD,E,VS1,VG,EX,14408689,202206,5910.0,1.00,5910.00,6.17,6.23,64.9,4.04327


In [25]:
# теперь сконкатенируем полученый датафрейм с той частью,
# которая не изменялась по высоте

without_y_changes = df_three[~df_three['id'].isin(list_y_index)]

df_three = without_y_changes.merge(cleared_y_changes, how='outer') \
           .sort_values(by=['id','date'])

df_three

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
60685,NONE,EX,RD,D,VVS2,EX,EX,19423,202205,13970.0,1.05,13304.76,6.67,6.64,59.0,3.91760
29314,NONE,EX,RD,D,VVS2,EX,EX,19449,202205,16850.0,1.17,14401.71,6.79,6.75,62.0,4.18500
40443,NONE,EX,RD,D,VVS2,EX,EX,19449,202207,17550.0,1.17,15000.00,6.79,6.75,62.0,4.18500
64627,FNT,VG,RD,D,VS1,VG,G,19501,202205,34980.0,2.00,17490.00,8.01,7.92,62.0,4.91040
64084,NONE,VG,EM SQ,E,VS2,NaN,EX,19504,202205,23140.0,2.01,11512.44,7.03,6.70,73.0,4.89100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,NONE,VG,EM,D,VS1,NaN,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
11046,MED,EX,EM,H,SI2,NaN,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975
11047,MED,EX,EM,H,SI2,NaN,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975
10636,NONE,EX,PR,G,VVS2,EX,EX,15412954,202207,7100.0,1.20,5916.67,5.80,5.76,72.4,4.17024


In [26]:
# проверим остались те объекты, чьи изменения были по каратам

list_carat_index = stat[stat['carat'] > 0].index

df_three[df_three['id'].isin(list_carat_index)].duplicated('id').sum()

0

In [27]:
# Также проверим z и x

list_zx_index = stat[(stat['z'] > 0) | (stat['x'] > 0)].index

df_three[df_three['id'].isin(list_zx_index)].duplicated('id').sum()

0

In [28]:
# и проверим list_depth_index

list_depth_index = stat[stat['depth_perc'] > 0].index

df_three[df_three['id'].isin(list_depth_index)].duplicated('id').sum()

0

Все изменения по 'z', 'x', 'carat' и 'depth_perc' удалились вместе с изменениями по высоте

In [29]:
# теперь в качестве цены возьмем последнее значение по времени

df_three = df_three.groupby('id').agg('last').reset_index()

# соединяем результаты

df = pd.concat([df_one, df_three]).reset_index(drop=True) 
df

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
0,NONE,EX,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570
1,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180
2,NONE,VG,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572
3,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040
4,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161985,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
161986,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
161987,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
161988,MED,EX,EM,H,SI2,None,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


Цены на бриллианты достаточно сильно меняются, попробуйте вычислить коэффициент инфляции и привести цены к последнему месяцу. Стоит учесть, что цены на все бриллианты не изменяются синхронно, то есть изменение в определенных группах может быть разным.

*Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Подправьте цены на эти коэффициенты и создайте новую переменную.*

In [30]:
# определим по каким группам отпределять инфляцию

categories = ['symmetry', 'shape', 'color', 'clarity', 'polish', 'carat'] 

# fluor и cut содержат нулевые значения, поэтому их не включаем в группы

In [31]:
df_index = df.groupby(['date'] + categories)[['price_per_carat']] \
           .mean().reset_index() # индекс цен

date_max = df_index['date'].max()

In [32]:
# сопоставляем группы с максимальной датой

df_index_max = df_index.merge(df_index.query('date == @date_max')
                          .iloc[:, 1:]
                          .rename(columns={'price_per_carat': 'price_per_carat_max'}),
                          on=categories, how='outer')
df_index_max

,date,symmetry,shape,color,clarity,polish,carat,price_per_carat,price_per_carat_max
0,202205,EX,CUSH,D,I1,EX,0.90,1455.56,NaN
1,202205,EX,CUSH,D,I1,EX,1.00,2400.00,NaN
2,202206,EX,CUSH,D,I1,EX,1.00,2450.00,NaN
3,202205,EX,CUSH,D,IF,EX,0.81,4185.19,NaN
4,202205,EX,CUSH,D,IF,EX,0.82,5426.83,NaN
...,...,...,...,...,...,...,...,...,...
63999,202207,VG,RD,K,VVS1,VG,1.30,5500.00,5500.00
64000,202207,VG,RD,K,VVS1,VG,1.52,6434.21,6434.21
64001,202207,VG,RD,K,VVS2,EX,0.95,4705.26,4705.26
64002,202207,VG,RD,K,VVS2,EX,1.72,6616.28,6616.28


Но у нас есть ещё промежуточное значение даты - 202206, где могло быть последнее известное повышение цены по группам. То есть к 202206 месяцу цена поднялась, а к 202207 нет, но беря только по 202207 мы получим NaN, хотя возможно изменение на самом деле было, но только в 202206 месяце.

In [33]:
df['date'].unique()

array([202205, 202207, 202206])

In [34]:
# Сопоставим группы по 202206 месяцу

df_index_202206 = df_index.merge(df_index.query('date == 202206')
                                 .iloc[:, 1:]
                                 .rename(columns={'price_per_carat': 'price_per_carat_max'}),
                                 on=categories, how='outer')
df_index_202206

,date,symmetry,shape,color,clarity,polish,carat,price_per_carat,price_per_carat_max
0,202205,EX,CUSH,D,I1,EX,0.90,1455.56,NaN
1,202205,EX,CUSH,D,I1,EX,1.00,2400.00,2450.0
2,202206,EX,CUSH,D,I1,EX,1.00,2450.00,2450.0
3,202205,EX,CUSH,D,IF,EX,0.81,4185.19,NaN
4,202205,EX,CUSH,D,IF,EX,0.82,5426.83,NaN
...,...,...,...,...,...,...,...,...,...
63999,202207,VG,RD,K,VVS1,VG,1.30,5500.00,NaN
64000,202207,VG,RD,K,VVS1,VG,1.52,6434.21,NaN
64001,202207,VG,RD,K,VVS2,EX,0.95,4705.26,NaN
64002,202207,VG,RD,K,VVS2,EX,1.72,6616.28,NaN


In [35]:
# Теперь выберем не пропущенные значения в df_index_max,
# они приоритетней чем за 202206, поскольку после 202206 цена могла снова измениться

df_index_max = df_index_max[df_index_max['price_per_carat_max'] \
                .notna()]
df_index_max

,date,symmetry,shape,color,clarity,polish,carat,price_per_carat,price_per_carat_max
7,202205,EX,CUSH,D,IF,EX,1.01,8004.950000,7267.33
8,202206,EX,CUSH,D,IF,EX,1.01,7323.430000,7267.33
9,202207,EX,CUSH,D,IF,EX,1.01,7267.330000,7267.33
28,202205,EX,CUSH,D,SI1,EX,1.50,7201.111667,5883.33
29,202206,EX,CUSH,D,SI1,EX,1.50,7566.665000,5883.33
...,...,...,...,...,...,...,...,...,...
63999,202207,VG,RD,K,VVS1,VG,1.30,5500.000000,5500.00
64000,202207,VG,RD,K,VVS1,VG,1.52,6434.210000,6434.21
64001,202207,VG,RD,K,VVS2,EX,0.95,4705.260000,4705.26
64002,202207,VG,RD,K,VVS2,EX,1.72,6616.280000,6616.28


In [36]:
# выберем теперь те индексы df_index_max, которые были с пропусками, но уже
# найдём их за 202206 месяц

df_index_202206 = df_index_202206[~df_index_202206.index.isin(df_index_max.index)]
df_index_202206 

,date,symmetry,shape,color,clarity,polish,carat,price_per_carat,price_per_carat_max
0,202205,EX,CUSH,D,I1,EX,0.90,1455.56,NaN
1,202205,EX,CUSH,D,I1,EX,1.00,2400.00,2450.00
2,202206,EX,CUSH,D,I1,EX,1.00,2450.00,2450.00
3,202205,EX,CUSH,D,IF,EX,0.81,4185.19,NaN
4,202205,EX,CUSH,D,IF,EX,0.82,5426.83,NaN
...,...,...,...,...,...,...,...,...,...
53175,202206,VG,RD,K,VVS2,EX,2.02,5232.67,5232.67
53176,202206,VG,RD,K,VVS2,EX,2.56,8445.31,8445.31
53177,202206,VG,RD,K,VVS2,VG,0.95,4052.63,4052.63
53178,202206,VG,RD,K,VVS2,VG,1.70,6035.29,6035.29


In [37]:
# сконкатенируем два датафрейма

df_index = pd.concat([df_index_202206, df_index_max])
df_index

,date,symmetry,shape,color,clarity,polish,carat,price_per_carat,price_per_carat_max
0,202205,EX,CUSH,D,I1,EX,0.90,1455.56,NaN
1,202205,EX,CUSH,D,I1,EX,1.00,2400.00,2450.00
2,202206,EX,CUSH,D,I1,EX,1.00,2450.00,2450.00
3,202205,EX,CUSH,D,IF,EX,0.81,4185.19,NaN
4,202205,EX,CUSH,D,IF,EX,0.82,5426.83,NaN
...,...,...,...,...,...,...,...,...,...
63999,202207,VG,RD,K,VVS1,VG,1.30,5500.00,5500.00
64000,202207,VG,RD,K,VVS1,VG,1.52,6434.21,6434.21
64001,202207,VG,RD,K,VVS2,EX,0.95,4705.26,4705.26
64002,202207,VG,RD,K,VVS2,EX,1.72,6616.28,6616.28


In [38]:
# вычисляем инфляцию

df_index['inflation'] = df_index['price_per_carat_max'] \
                        / df_index['price_per_carat']
df_index

,date,symmetry,shape,color,clarity,polish,carat,price_per_carat,price_per_carat_max,inflation
0,202205,EX,CUSH,D,I1,EX,0.90,1455.56,NaN,NaN
1,202205,EX,CUSH,D,I1,EX,1.00,2400.00,2450.00,1.020833
2,202206,EX,CUSH,D,I1,EX,1.00,2450.00,2450.00,1.000000
3,202205,EX,CUSH,D,IF,EX,0.81,4185.19,NaN,NaN
4,202205,EX,CUSH,D,IF,EX,0.82,5426.83,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
63999,202207,VG,RD,K,VVS1,VG,1.30,5500.00,5500.00,1.000000
64000,202207,VG,RD,K,VVS1,VG,1.52,6434.21,6434.21,1.000000
64001,202207,VG,RD,K,VVS2,EX,0.95,4705.26,4705.26,1.000000
64002,202207,VG,RD,K,VVS2,EX,1.72,6616.28,6616.28,1.000000


Пропуски в inflation можно заменить на 1, поскольку это объекты, чья цена с 202205 месяца не менялась

In [39]:
df_index['inflation'] = df_index['inflation'].fillna(1)

In [40]:
# соединяем всё в одной таблице

df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']], 
                        on=['date'] + categories, how='left')
df_with_inf

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation
0,NONE,EX,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570,0.993874
1,NONE,VG,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,1.000000
2,NONE,VG,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572,1.000000
3,MED,EX,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.055916
4,NONE,EX,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.023450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161985,NONE,EX,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000
161986,MED,EX,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,1.000000
161987,NONE,VG,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,1.000000
161988,MED,EX,EM,H,SI2,None,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975,1.000000


In [41]:
df_with_inf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161990 entries, 0 to 161989
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fluor            161983 non-null  object 
 1   symmetry         161990 non-null  object 
 2   shape            161990 non-null  object 
 3   color            161990 non-null  object 
 4   clarity          161990 non-null  object 
 5   cut              109426 non-null  object 
 6   polish           161990 non-null  object 
 7   id               161990 non-null  int64  
 8   date             161990 non-null  int64  
 9   price            161990 non-null  float64
 10  carat            161990 non-null  float64
 11  price_per_carat  161990 non-null  float64
 12  z                161990 non-null  float64
 13  x                161990 non-null  float64
 14  depth_perc       161990 non-null  float64
 15  y                161990 non-null  float64
 16  inflation        161990 non-null  floa

### Задание 2: Модель

Определите функцию потерь (MSE или MAE) и аргументируйте выбор.
Попробуйте сделать baseline.
Используйте LableEncoder для категориальных фичей и **постройте линейную модель**.

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

# определим X и y

X = df_with_inf.drop(columns=["price_per_carat"])
y = df_with_inf["price_per_carat"]

In [43]:
# определим список категориальных признаков в переменную

obj_columns = X.select_dtypes(include='object').columns
obj_columns

Index(['fluor', 'symmetry', 'shape', 'color', 'clarity', 'cut', 'polish'], dtype='object')

In [45]:
# закодируем каждый категориальный признак

le = LabelEncoder()

X['fluor'] = le.fit_transform(X[obj_columns[0]])
X['symmetry'] = le.fit_transform(X[obj_columns[1]])
X['shape'] = le.fit_transform(X[obj_columns[2]])
X['color'] = le.fit_transform(X[obj_columns[3]])
X['clarity'] = le.fit_transform(X[obj_columns[4]])
X['cut'] = le.fit_transform(X[obj_columns[5]])
X['polish'] = le.fit_transform(X[obj_columns[6]])

X

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,z,x,depth_perc,y,inflation
0,2,0,7,0,4,4,0,14224443,202205,9120.0,1.51,10.67,6.23,59.0,3.67570,0.993874
1,2,3,1,7,4,4,2,12902293,202205,1390.0,0.91,6.30,4.58,71.0,3.25180,1.000000
2,2,3,7,6,5,4,2,12771762,202207,15990.0,2.27,11.66,7.26,62.2,4.51572,1.000000
3,1,0,9,6,4,0,0,14584509,202205,3700.0,1.01,6.37,6.42,62.0,3.98040,1.055916
4,2,0,9,3,3,0,0,14773748,202206,7170.0,1.00,6.30,6.35,63.7,4.04495,1.023450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161985,2,0,9,5,3,0,0,15409696,202207,6040.0,1.01,6.46,6.49,60.8,3.94592,1.000000
161986,1,0,5,4,8,3,0,15410303,202207,6620.0,1.01,7.76,5.40,65.0,3.51000,1.000000
161987,2,3,1,0,5,3,0,15410668,202207,14750.0,1.50,7.84,5.43,65.8,3.57294,1.000000
161988,1,0,1,4,4,3,0,15410717,202207,48230.0,4.53,11.61,7.57,67.5,5.10975,1.000000


In [46]:
# теперь разделим наши данные на тренировочную и тестовую выборки

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [47]:
# обучим модель и сделаем первое предсказание

from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_valid)

print('MSE:', mean_squared_error(y_valid, y_pred))
print('MAE:', mean_absolute_error(y_valid, y_pred))

Казалось бы что лучше использовать MAE, поскольку ошибка возведённая в квадрат становится слишком большой для адекватной интерпретации, но MSE тоже может сослужить хорошую службу, поскольку в динамике способно показать, насколько больше или меньше стало предсказанных значений с большим отрывом от истинного.

Теперь попробуйте OHE или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с пропусками (обратите внимание на то, что у fluor возможен пропуск значения, а возможно отсутствие флуоресценции)

Покажите, насколько получилось улучшить результат.

In [49]:
! pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 3.3 MB/s eta 0:00:00


In [50]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

from typing import Tuple, List


def OHE(df: pd.DataFrame, columns: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    """
    Вместо это функции можно использовать и функционал pandas
    """
    index = df.index
    one = OneHotEncoder(sparse=False, categories='auto')
    ohe = one.fit_transform(df[columns])
    col_names = one.get_feature_names(input_features = columns)
    df = df.drop(columns, axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(ohe, columns=col_names)], axis = 1)
    df = df.set_index(index)
    return (df, col_names)

scaler = StandardScaler() # Помните, что на тесте делаем только transform

In [51]:
# обновим X и y

X = df_with_inf.drop(columns=["price_per_carat"])
y = df_with_inf["price_per_carat"]

In [52]:
# посмотрим на результат TargetEncoder

te = TargetEncoder()

te.fit_transform(X, y)

,fluor,symmetry,shape,color,clarity,cut,polish,id,date,price,carat,z,x,depth_perc,y,inflation
0,7152.937471,7932.267766,6347.756611,9524.053377,5110.553986,6032.762692,7243.503752,14224443,202205,9120.0,1.51,10.67,6.23,59.0,3.67570,0.993874
1,7152.937471,5692.449762,6006.891297,4315.303994,5110.553986,6032.762692,5460.973964,12902293,202205,1390.0,0.91,6.30,4.58,71.0,3.25180,1.000000
2,7152.937471,5692.449762,6347.756611,5154.487992,7954.084133,6032.762692,5460.973964,12771762,202207,15990.0,2.27,11.66,7.26,62.2,4.51572,1.000000
3,6864.972148,7932.267766,8034.898077,5154.487992,5110.553986,8221.660706,7243.503752,14584509,202205,3700.0,1.01,6.37,6.42,62.0,3.98040,1.055916
4,7152.937471,7932.267766,8034.898077,7318.832548,6245.672110,8221.660706,7243.503752,14773748,202206,7170.0,1.00,6.30,6.35,63.7,4.04495,1.023450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161985,7152.937471,7932.267766,8034.898077,6101.132061,6245.672110,8221.660706,7243.503752,15409696,202207,6040.0,1.01,6.46,6.49,60.8,3.94592,1.000000
161986,6864.972148,7932.267766,6115.560653,6904.559903,8387.077312,6032.762692,7243.503752,15410303,202207,6620.0,1.01,7.76,5.40,65.0,3.51000,1.000000
161987,7152.937471,5692.449762,6006.891297,9524.053377,7954.084133,6032.762692,7243.503752,15410668,202207,14750.0,1.50,7.84,5.43,65.8,3.57294,1.000000
161988,6864.972148,7932.267766,6006.891297,6904.559903,5110.553986,6032.762692,7243.503752,15410717,202207,48230.0,4.53,11.61,7.57,67.5,5.10975,1.000000


In [53]:
# а теперь на результат OHE

OHE(X, obj_columns)[0]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,id,date,price,carat,z,x,depth_perc,y,inflation,fluor_FNT,...,clarity_VVS1,clarity_VVS2,cut_EX,cut_G,cut_VG,cut_None,cut_nan,polish_EX,polish_G,polish_VG
0,14224443,202205,9120.0,1.51,10.67,6.23,59.0,3.67570,0.993874,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,12902293,202205,1390.0,0.91,6.30,4.58,71.0,3.25180,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,12771762,202207,15990.0,2.27,11.66,7.26,62.2,4.51572,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,14584509,202205,3700.0,1.01,6.37,6.42,62.0,3.98040,1.055916,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,14773748,202206,7170.0,1.00,6.30,6.35,63.7,4.04495,1.023450,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161985,15409696,202207,6040.0,1.01,6.46,6.49,60.8,3.94592,1.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
161986,15410303,202207,6620.0,1.01,7.76,5.40,65.0,3.51000,1.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
161987,15410668,202207,14750.0,1.50,7.84,5.43,65.8,3.57294,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
161988,15410717,202207,48230.0,4.53,11.61,7.57,67.5,5.10975,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [54]:
# NaN значения тоже были отдельное выделены

OHE(X, obj_columns)[0].columns

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Index(['id', 'date', 'price', 'carat', 'z', 'x', 'depth_perc', 'y',
       'inflation', 'fluor_FNT', 'fluor_MED', 'fluor_NONE', 'fluor_STG',
       'fluor_None', 'fluor_nan', 'symmetry_EX', 'symmetry_G', 'symmetry_No',
       'symmetry_VG', 'shape_CUSH', 'shape_EM', 'shape_EM SQ', 'shape_HS',
       'shape_MQ', 'shape_OV', 'shape_PR', 'shape_PS', 'shape_RAD', 'shape_RD',
       'color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
       'color_J', 'color_K', 'color_L', 'color_M', 'clarity_FL', 'clarity_I1',
       'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'cut_EX', 'cut_G',
       'cut_VG', 'cut_None', 'cut_nan', 'polish_EX', 'polish_G', 'polish_VG'],
      dtype='object')

In [55]:
# посмотрим на работу OHE при попомощи встроенного метода в Pandas также с NaN

pd.get_dummies(X, dummy_na=True)

,id,date,price,carat,z,x,depth_perc,y,inflation,fluor_FNT,...,clarity_VVS2,clarity_nan,cut_EX,cut_G,cut_VG,cut_nan,polish_EX,polish_G,polish_VG,polish_nan
0,14224443,202205,9120.0,1.51,10.67,6.23,59.0,3.67570,0.993874,0,...,0,0,0,0,0,1,1,0,0,0
1,12902293,202205,1390.0,0.91,6.30,4.58,71.0,3.25180,1.000000,0,...,0,0,0,0,0,1,0,0,1,0
2,12771762,202207,15990.0,2.27,11.66,7.26,62.2,4.51572,1.000000,0,...,0,0,0,0,0,1,0,0,1,0
3,14584509,202205,3700.0,1.01,6.37,6.42,62.0,3.98040,1.055916,0,...,0,0,1,0,0,0,1,0,0,0
4,14773748,202206,7170.0,1.00,6.30,6.35,63.7,4.04495,1.023450,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161985,15409696,202207,6040.0,1.01,6.46,6.49,60.8,3.94592,1.000000,0,...,0,0,1,0,0,0,1,0,0,0
161986,15410303,202207,6620.0,1.01,7.76,5.40,65.0,3.51000,1.000000,0,...,1,0,0,0,0,1,1,0,0,0
161987,15410668,202207,14750.0,1.50,7.84,5.43,65.8,3.57294,1.000000,0,...,0,0,0,0,0,1,1,0,0,0
161988,15410717,202207,48230.0,4.53,11.61,7.57,67.5,5.10975,1.000000,0,...,0,0,0,0,0,1,1,0,0,0


У встроенного метода на 3 признака больше. Сложно сказать, стоит ли использовать для NaN отдельный признак, но для того, чтобы не смешивать определёность с неопределёностью, наверно, следует их вынести отдельно. Хотя лучше их в большинстве случаев заполнять. Но мы пока опустим этот момент. 

Выберем встроенный метод OHE, поскольку он работает без предупреждений. А также будем использовать атрибут first_drop=True, чтобы чуть-чуть уменьшить кол-во признаков

In [56]:
# Сначала посмотрим как отработает модель при TargetEncoder

X_train, X_valid, y_train, y_valid = train_test_split(te.fit_transform(X, y),
                                                      y, test_size=0.2, 
                                                      random_state=13)

lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_valid)

print('MSE:', mean_squared_error(y_valid, y_pred))
print('MAE:', mean_absolute_error(y_valid, y_pred))

MSE:  21610041.741742108
MAE:  1044.7361396199044


MAE стало меньше чем при LabelEncoder, зато MSE стало больше. Это говорит нам о том, что количество значений, экстремально отличающихся от истинного, стало больше или они стали больше ещё сами по себе, или то и другое одновременно.

In [57]:
# Теперь посмотрим как отработает модель при OHE от Pandas

X_train, X_valid, y_train, y_valid = train_test_split(pd.get_dummies(X, 
                                                                     dummy_na=True, 
                                                                     drop_first=True),
                                                      y, test_size=0.2, 
                                                      random_state=13)

lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_valid)

print('MSE:', mean_squared_error(y_valid, y_pred))
print('MAE:', mean_absolute_error(y_valid, y_pred))

MSE:  19629177.743484177
MAE:  893.6506581332568


При OHE результат стал лучше, MSE и MAE уменьшились. Но давайте дадим шанс TargetEncoder путём заполнением пропусков и нормализации данных

In [58]:
# заполним по моде

X['fluor'] = X['fluor'].fillna(X['fluor'].mode()[0]) # NONE - отсутствие флуоресценции
X['cut'] = X['cut'].fillna(X['cut'].mode()[0])

In [59]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161990 entries, 0 to 161989
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   fluor       161990 non-null  object 
 1   symmetry    161990 non-null  object 
 2   shape       161990 non-null  object 
 3   color       161990 non-null  object 
 4   clarity     161990 non-null  object 
 5   cut         161990 non-null  object 
 6   polish      161990 non-null  object 
 7   id          161990 non-null  int64  
 8   date        161990 non-null  int64  
 9   price       161990 non-null  float64
 10  carat       161990 non-null  float64
 11  z           161990 non-null  float64
 12  x           161990 non-null  float64
 13  depth_perc  161990 non-null  float64
 14  y           161990 non-null  float64
 15  inflation   161990 non-null  float64
dtypes: float64(7), int64(2), object(7)
memory usage: 25.0+ MB


In [60]:
# Даём шанс TargetEncoder

X_train, X_valid, y_train, y_valid = train_test_split(te.fit_transform(X, y),
                                                      y, test_size=0.2, 
                                                      random_state=13)
scaler.fit(X_train, y_train)

X_train_normed = scaler.transform(X_train)
X_valid_normed = scaler.transform(X_valid)

lr_model = LinearRegression()

lr_model.fit(X_train_normed, y_train)

y_pred = lr_model.predict(X_valid_normed)

print('MSE:', mean_squared_error(y_valid, y_pred))
print('MAE:', mean_absolute_error(y_valid, y_pred))

MSE:  21562726.569152087
MAE:  1046.9149200606987


По MSE стало чуть лучше, а по MAE чуть хуже, но в целом чудо не произошло))

Тогда давайте улучшим нашу модель с OHE добавив заполненные пропуски и нормализацию

In [61]:
X_train, X_valid, y_train, y_valid = train_test_split(pd.get_dummies(X, 
                                                                     drop_first=True),
                                                      y, test_size=0.2, 
                                                      random_state=13)

scaler.fit(X_train, y_train)

X_train_normed = scaler.transform(X_train)
X_valid_normed = scaler.transform(X_valid)

lr_model = LinearRegression()

lr_model.fit(X_train_normed, y_train)

y_pred = lr_model.predict(X_valid_normed)

print('MSE:', mean_squared_error(y_valid, y_pred))
print('MAE:', mean_absolute_error(y_valid, y_pred))
print()
print('Разница по MSE с нормализацией и заполненными пропусками и без них:', 
      mean_squared_error(y_valid, y_pred) - 19629177.743484177)
print('Разница по MAE с нормализацией и заполненными пропусками и без них:', 
      mean_absolute_error(y_valid, y_pred) - 893.6506581332568)

MSE:  19629139.45048286
MAE:  893.6486230877922

Разница по MSE с нормализацией и заполненными пропусками и без них:  -38.29300131648779
Разница по MAE с нормализацией и заполненными пропусками и без них:  -0.0020350454645949867


Результат улучшился, но совсем незначительно

Сравните работу выбранной вами линейной модели с KNN

In [62]:
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor(n_neighbors=3)

# применим предыдущие данные с OHE, заполненными значениями и нормализацией, 
# поскольку для knn важна сопоставимость признаков

knn_model.fit(X_train_normed, y_train)

y_pred = knn_model.predict(X_valid_normed)

print('MSE:', mean_squared_error(y_valid, y_pred))
print('MAE:', mean_absolute_error(y_valid, y_pred))

MSE:  23390917.78427891
MAE:  785.5578402987838


MAE стало меньше чем при линейной регрессии, но вот MSE выросло. Это значит, что есть значения, которые сильно выбиваются от истинного. Возможно, можно подобрать более оптимальное кол-во "соседей" для лучшего результата в сравнении с ЛР, но тут knn уже работает не так быстро как линейная регрессия, поэтому такой подбор будет очень долгим.

Поэтому если неважны какие-то редкие значения, которые сильно отбиваются от истинного значения, то лучше выбирать MAE. Если же такие значения критичны, то MSE.

## Классификация

Загрузите датасет bodyPerformance

Описание:

https://www.kaggle.com/datasets/kukuroo3/body-performance-data

In [63]:
# from google.colab import drive
# drive.mount('/content/drive')

# path = 'drive/MyDrive/'

In [72]:
df = pd.read_csv('/content/bodyPerformance.csv')

In [73]:
df

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13393 entries, 0 to 13392
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      13393 non-null  float64
 1   gender                   13393 non-null  object 
 2   height_cm                13393 non-null  float64
 3   weight_kg                13393 non-null  float64
 4   body fat_%               13393 non-null  float64
 5   diastolic                13393 non-null  float64
 6   systolic                 13393 non-null  float64
 7   gripForce                13393 non-null  float64
 8   sit and bend forward_cm  13393 non-null  float64
 9   sit-ups counts           13393 non-null  float64
 10  broad jump_cm            13393 non-null  float64
 11  class                    13393 non-null  object 
dtypes: float64(10), object(2)
memory usage: 1.2+ MB


In [76]:
df.nunique()

age                          44
gender                        2
height_cm                   467
weight_kg                  1398
body fat_%                  527
diastolic                    89
systolic                    102
gripForce                   550
sit and bend forward_cm     528
sit-ups counts               81
broad jump_cm               245
class                         4
dtype: int64

### Задание 1: Определение гендера

Постройте модель, которая будет определять гендер.

Для этого определите две метрики качества, на которые вы будете смотреть. Аргументируйте свой выбор.

После чего преобразуйте категориальные переменные и постройте модель бинарной классификации на основе линейного классификатора и сравните с KNN.

P.S.: не забудте про нормализацию.

In [78]:
# преобразуем категориальные признаки

df_new = pd.get_dummies(df, drop_first=True)
df_new

,age,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,gender_M,class_B,class_C,class_D
0,27.0,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,1,0,1,0
1,25.0,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,1,0,0,0
2,31.0,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,1,0,1,0
3,32.0,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1,1,0,0
4,28.0,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,1,0,1,0
13389,21.0,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,1,0,0,1
13390,39.0,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,1,0,0,0
13391,64.0,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,0,0,0,1


In [79]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Наш признак gender после OHE стал называться gender_M

X = df_new.drop(columns=["gender_M"])
y = df_new["gender_M"]

In [80]:
# Проверим на сбалансированность классов

y.value_counts(normalize=True)

1    0.632196
0    0.367804
Name: gender_M, dtype: float64

Классы несбалансированные, поэтому при разбиение на тренировочную и тестовую выборки будем использовать стратификацию. 

Метрики качества выбраны F1 и ROC_AUC, поскольку они менее чувствительны к несбалансированности классов, но также рядом с ними будут использоваться Accuracy, Precision и Recall для общей наглядности.

In [88]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.8,
                                                      random_state=13, 
                                                      stratify=y)

# test_size был выбран 0.8 для доп. сложности

scaler = StandardScaler()

scaler.fit(X_train, y_train)

X_train_normed = scaler.transform(X_train)
X_valid_normed = scaler.transform(X_valid)

logreg_model = LogisticRegression()

logreg_model.fit(X_train_normed, y_train)

y_pred = logreg_model.predict(X_valid_normed)

print('Accuracy:', accuracy_score(y_valid, y_pred))
print('Precision:', precision_score(y_valid, y_pred))
print('Recall:', recall_score(y_valid, y_pred))
print('F1:', f1_score(y_valid, y_pred))
print('ROC_AUC:', roc_auc_score(y_valid, logreg_model.predict_proba(X_valid_normed)[:, 1]))

Accuracy:  0.9865608959402706
Precision:  0.9903846153846154
Recall:  0.9883377620312961
F1:  0.9893601300428551
ROC_AUC:  0.9980427649728986


Очень высокие показатели. Такие показатели были бы возможны при переобучении модели, но здесь скорей всего просто хорошие данные, поскольку если бы модель переобучилась, то она бы классифицировала почти все объекты как 1, и в итоге у нас то же Accuracy было бы ± равно доле первого класса (около 60%)

In [89]:
# теперь построим модель по knn

knnс_model = KNeighborsClassifier(n_neighbors=3)

knnс_model.fit(X_train_normed, y_train)

y_pred = knnс_model.predict(X_valid_normed)

print('Accuracy:', accuracy_score(y_valid, y_pred))
print('Precision:', precision_score(y_valid, y_pred))
print('Recall:', recall_score(y_valid, y_pred))
print('F1:', f1_score(y_valid, y_pred))
print('ROC_AUC:', roc_auc_score(y_valid, knnс_model.predict_proba(X_valid_normed)[:, 1]))

Accuracy:  0.9850676621558563
Precision:  0.9873268493958149
Recall:  0.9890758783584293
F1:  0.9882005899705015
ROC_AUC:  0.9943642074600955


In [90]:
# Сравним KNN с LR

print('Accuracy KNN - LR:', accuracy_score(y_valid, y_pred) - 0.9865608959402706)
print('Precision KNN - LR:', precision_score(y_valid, y_pred) - 0.9903846153846154)
print('Recall KNN - LR:', recall_score(y_valid, y_pred) - 0.9883377620312961)
print('F1 KNN - LR:', f1_score(y_valid, y_pred) - 0.9893601300428551)
print('ROC_AUC KNN - LR:', roc_auc_score(y_valid, 
      knnс_model.predict_proba(X_valid_normed)[:, 1]) - 0.9980427649728986)

Accuracy KNN - LR:  -0.0014932337844143495
Precision KNN - LR:  -0.003057765988800476
Recall KNN - LR:  0.0007381163271331648
F1 KNN - LR:  -0.0011595400723535265
ROC_AUC KNN - LR:  -0.0036785575128031045


В целом логистическая регрессия даёт результаты чуть лучше

### Задание 2: Определение класса

Теперь опробуем построить модель, которая будет предсказывать class физической формы.

Для этого определите метрики качества для задачи мультиклассификации (аргументируйте выбор).

Постройте модель мультиклассовой классификации на основе линейного классификатора и сравните с KNN.

In [93]:
df

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


In [98]:
le = LabelEncoder()

X = df.drop(columns=["class"])
X['gender'] = le.fit_transform(X['gender']) # также преобразовали gender

y = pd.Series(le.fit_transform(df["class"])) # и преобразовали сразу таргет

In [99]:
# проверка на сбалансированность классов

y.value_counts(normalize=True)

2    0.250056
3    0.250056
0    0.249981
1    0.249907
dtype: float64

Классы хорошо сбалансированны

In [133]:
# Разбиваем данные и прогнозируем

# опять же test_size=0.8, поскольку данные очень хорошие, можно и усложнить

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.8, 
                                                      random_state=13)

scaler = StandardScaler()

scaler.fit(X_train, y_train)

X_train_normed = scaler.transform(X_train)
X_valid_normed = scaler.transform(X_valid)

logreg_model = LogisticRegression()

logreg_model.fit(X_train_normed, y_train)

y_pred = logreg_model.predict(X_valid_normed)

Чтобы оценить многоклассовую модель, которая при этом хорошо сбалансирована, можно бинарно оценить каждый предсказанный класс, а затем усреднить их. Это можно сделать с помощью Macro Precision

In [134]:
print('Macro Precision:', precision_score(y_valid, y_pred, average='macro'))

# сравним также с Micro Precision
print('Micro Precision:', precision_score(y_valid, y_pred, average='micro'))

Macro Precision: 0.6070414247275004
Micro Precision: 0.6101726551563229


Попробуем похожее с ROC_AUC

In [135]:
print('OvO:', roc_auc_score(y_valid, logreg_model.predict_proba(X_valid_normed), 
                            multi_class='ovo'))
print('OvR:', roc_auc_score(y_valid, logreg_model.predict_proba(X_valid_normed), 
                            multi_class='ovr'))

OvO: 0.8495507818560902
OvR: 0.8492946691919796


А как с F1?

In [136]:
print('Macro F1:', f1_score(y_valid, y_pred, average='macro'))
print('Micro F1:', f1_score(y_valid, y_pred, average='micro'))

Macro F1: 0.6078236923560951
Micro F1: 0.6101726551563229


Precision макро и микро уровни почти совпадают с F1. А вот ROC_AUC нет.

Одного Precision мало, хочется взять среднегармоническое между ним и Recall, поэтому дальше будем оценивать по F1 (макро и микро), а также по ROC_AUC (OvO и OvR), как альтернатива.

In [137]:
# теперь спрогнозируем по KNN

knnс_model = KNeighborsClassifier(n_neighbors=3)

knnс_model.fit(X_train_normed, y_train)

y_pred = knnс_model.predict(X_valid_normed)

print('Macro F1:', f1_score(y_valid, y_pred, average='macro'))
print('Micro F1:', f1_score(y_valid, y_pred, average='micro'))
print('OvO:', roc_auc_score(y_valid, knnс_model.predict_proba(X_valid_normed), 
                            multi_class='ovo'))
print('OvR:', roc_auc_score(y_valid, knnс_model.predict_proba(X_valid_normed), 
                            multi_class='ovr'))

Macro F1: 0.5330176685281305
Micro F1: 0.5334577694820345
OvO: 0.7668312892492075
OvR: 0.7666385688337177


In [138]:
# Сравним с KNN с логистической регрессией

print('Macro F1 KNN - LR:', f1_score(y_valid, y_pred, average='macro') - 0.6078236923560951)
print('Micro F1 KNN - LR:', f1_score(y_valid, y_pred, average='micro') - 0.6101726551563229)
print('OvO KNN - LR:', roc_auc_score(y_valid, knnс_model.predict_proba(X_valid_normed), 
                            multi_class='ovo') - 0.8495507818560902)
print('OvR KNN - LR:', roc_auc_score(y_valid, knnс_model.predict_proba(X_valid_normed), 
                            multi_class='ovr') - 0.8492946691919796)

Macro F1 KNN - LR: -0.07480602382796464
Micro F1 KNN - LR: -0.07671488567428841
OvO KNN - LR: -0.08271949260688272
OvR KNN - LR: -0.08265610035826187


Опять же, логистическая модель снова отработала чуть лучше чем KNN